In [0]:
dbutils.library.installPyPI('geopandas')
dbutils.library.installPyPI('descartes')

dbutils.library.restartPython()

In [0]:

from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import MultiLineString, Polygon, LineString
#from shapely.ops import polygonize
#from shapely.wkt import dumps, loads
from shapely.ops import cascaded_union, transform, Point
import shapely.speedups
shapely.speedups.enable() 

In [0]:
gdf  = gpd.read_file('/dbfs/mnt/DataSources/libraries/UAE_Corrected_Communities.shp')
gdf.head()


,zone_id,zone_code,zone_name_,city_id,city_code,state_code,WKT,geometry
0,10101,101,NAKHLAT DEIRA (PALM DEIRA),DXB,1,3,POLYGON ((332711.122756265569478 2800232.94070...,"POLYGON ((55.33816 25.30929, 55.33715 25.30778..."
1,10111,111,AL CORNICHE,DXB,1,3,POLYGON ((329833.092213331721723 2797089.23060...,"POLYGON ((55.30997 25.28059, 55.30992 25.28013..."
2,10112,112,AL RAS,DXB,1,3,POLYGON ((328560.758438463555649 2795737.78568...,"POLYGON ((55.29751 25.26824, 55.29792 25.26720..."
3,10113,113,AL DAGHAYA,DXB,1,3,POLYGON ((329044.756515259796288 2796178.98587...,"POLYGON ((55.30226 25.27228, 55.30238 25.27161..."
4,10114,114,AL BUTEEN,DXB,1,3,POLYGON ((329003.271672606817447 2795818.48925...,"POLYGON ((55.30189 25.26902, 55.30205 25.26892..."


In [0]:
df = spark.table("home_locations.dubai_home_locations_2020")
df.show()

+-----------+------------------+------------------+
customer_id| home_lat| home_lon|
+-----------+------------------+------------------+
 1| 25.05289599999999| 55.17976878378377|
 4| 25.3656744|55.512346400000006|
 8| 25.07717245| 55.14740102499999|
 19|25.234772611111115| 55.26237305555556|
 23|24.309113199999995|54.565763100000005|
 24|25.197216599999997|55.277686900000006|
 28| 25.0919406| 55.24390459999999|
 29|25.032583439153434| 55.2188647169312|
 39|25.125288454545455| 55.12219390909092|
 54| 25.07948254444444|55.121863099999985|
 55| 24.91680015| 55.00658476666666|
 57|25.078360375000003| 55.122894375|
 59|25.226758489361703| 55.2574980425532|
 60|25.156625896551716| 55.37541337931035|
 82|25.044728000000003| 55.24913366666666|
 87|24.362330041666667|54.547352458333336|
 88|25.053017354285718|55.207603471428584|
 93|24.988806896000007| 55.20271266799998|
 97| 25.03116907692307|55.217813538461535|
 105|25.209961362068963|55.419539603448264|
+-----------+------------------+------------------+
only showing top 20 rows

In [0]:
df=df.drop_duplicates()
df.show()

+-----------+------------------+------------------+
customer_id| home_lat| home_lon|
+-----------+------------------+------------------+
 9529640| 25.07553444736842| 55.13057042105263|
 9682741| 25.21399585227273|55.276242181818176|
 9704856|25.093936470588236|55.148172882352945|
 9797994| 24.48807825| 54.60830725|
 9813972|25.299665122222223| 55.37710997777778|
 9835561| 25.26015014285715| 55.28896678571429|
 9906537|24.998184766666668| 55.30040862222223|
 9942300| 25.33476339285715|55.373846035714294|
 9952796|25.102451571428563|55.177307821428556|
 9987650|25.059632406638976| 55.18380765975103|
 9988106| 25.1105092| 55.1368431|
 9992773|25.224031644444445| 55.42527475555555|
 10046733|24.355396727272726|54.659713636363634|
 10110469|25.050022538461533| 55.16675576923077|
 10147557|25.157791285714286|55.299752285714284|
 10154572|25.079652486111108| 55.13472156944445|
 10744598| 25.26225260666667| 55.32229454000001|
 10771709|25.090617037037035| 55.17095162962963|
 10772591| 25.26317706666667| 55.324654|
 10772679| 25.08858575| 55.157465|
+-----------+------------------+------------------+
only showing top 20 rows

In [0]:
sc.broadcast(gdf)

Out[4]: <pyspark.broadcast.Broadcast at 0x7f3eee9fa940>

In [0]:
def find_borough(home_lat, home_lon): 
    mgdf = gdf.apply(lambda x: (x['zone_name_']) if x['geometry'].intersects(Point(float(home_lon),float(home_lat))) else None, axis=1)
    idx = mgdf.first_valid_index()
    first_valid_value = mgdf.loc[idx] if idx is not None else None
    return first_valid_value

In [0]:
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType,DecimalType
find_borough_udf = udf(find_borough, StringType())
#find_borough_udf = udf(find_borough, IntegerType())

In [0]:
# test the function
find_borough( 25.264545,55.304037)

Out[7]: 'AL HAMRIYA'

In [0]:
from pyspark.sql.functions import *
df = df.withColumn("Zone_Neighbourhood", find_borough_udf(col("home_lat"),col("home_lon")))
df.show()

+-----------+------------------+------------------+--------------------+
customer_id| home_lat| home_lon| Zone_Neighbourhood|
+-----------+------------------+------------------+--------------------+
 9529640| 25.07553444736842| 55.13057042105263| MARSA DUBAI|
 9682741| 25.21399585227273|55.276242181818176| TRADE CENTER FIRST|
 9704856|25.093936470588236|55.148172882352945| MARSA DUBAI|
 9797994| 24.48807825| 54.60830725| YAS ISLAND|
 9813972|25.299665122222223| 55.37710997777778| AL NAHDA|
 9835561| 25.26015014285715| 55.28896678571429| AL SOUQ AL KABEER|
 9906537|24.998184766666668| 55.30040862222223|AL YALAYIS 2 (DUB...|
 9942300| 25.33476339285715|55.373846035714294| AL MAZAJ THIRD|
 9952796|25.102451571428563|55.177307821428556|AL THANYAH FIRST ...|
 9987650|25.059632406638976| 55.18380765975103|AL THANYAH FOURT...|
 9988106| 25.1105092| 55.1368431|NAKHLAT JUMEIRA (...|
 9992773|25.224031644444445| 55.42527475555555| MIRDIF|
 10046733|24.355396727272726|54.659713636363634| AL SHAWAMEKH|
 10110469|25.050022538461533| 55.16675576923077|AL THANYAH FIFTH ...|
 10147557|25.157791285714286|55.299752285714284|NADD AL SHIBA FIR...|
 10154572|25.079652486111108| 55.13472156944445| MARSA DUBAI|
 10744598| 25.26225260666667| 55.32229454000001| AL MURAQQABAT|
 10771709|25.090617037037035| 55.17095162962963|AL THANYAH THIRD ...|
 10772591| 25.26317706666667| 55.324654| AL MURAQQABAT|
 10772679| 25.08858575| 55.157465|AL THANYAH THIRD ...|
+-----------+------------------+------------------+--------------------+
only showing top 20 rows

In [0]:
df_filter_1=df[df.Zone_Neighbourhood=='MARSA DUBAI']
df_filter_2=df[df.Zone_Neighbourhood=='AL HEBIAH SECOND (DUBAI LAND)']


In [0]:
df_user=spark.table('recommendations.all_loc_users_info')
df_user.show()

+-------+--------------------+-----------+------+-------------------+---------------+----+
user_id| email|nationality|gender| registration_date|user_ent_status| cor|
+-------+--------------------+-----------+------+-------------------+---------------+----+
 29853|euromel_lang@yaho...| null| null|2011-06-13 03:00:00| 1| AE|
4682623| jetclu84@gmail.com| null| null|2018-05-23 08:42:40| 1| AE|
3146905| aaaaaa@vvvv.com| null| null|2017-08-15 17:04:20| 1| BH|
4058165| wouter@pentz.co.za| null| null|2018-02-11 17:56:46| 1| ZA|
2189261|jan.tezcanli@eric...| null| null|2016-12-08 15:07:47| 1| SE|
4295088|sherylcurilan@gma...| null| null|2018-03-27 12:27:56| 1| AE|
5262968|9178430228706263@...| null| null|2018-07-19 23:29:25| 1| AE|
7839261|faranha.bhula@sta...| null| null|2019-05-10 14:54:14| 1| ZA|
 227569|yianosoly@hotmail...| null| null|2014-07-17 17:10:00| 4| AE|
3522718|bryanmagat05@gmai...| null| null|2017-11-16 19:28:46| 1| AE|
2840755|bskshsisna@gmail.com| null| M|2017-05-06 02:17:23| 1| BH|
3380731|appumthomas@gmail...| null| null|2017-10-21 17:03:02| 1| AE|
2761906| eamonn@mehc.ae| null| null|2017-04-11 13:34:19| 1| AE|
 334762| aaronlow@gmail.com| null| null|2015-01-17 22:45:23| 1| SG|
4120228|generpereabras03@...| null| null|2018-02-23 17:53:29| 1| AE|
4463870| 334931791@qq.com| null| null|2018-04-23 17:29:27| 1| AS|
3367132|taonamuziwi@gmail...| null| null|2017-10-18 21:58:27| 1| QA|
7284665| null| null| null|2019-03-06 22:05:25| 1|null|
6128369|9278492006114096@...| null| null|2018-12-15 18:33:49| 1| AE|
1884834|aeatkins92@gmail.com| null| null|2016-09-25 19:45:59| 1| GB|
+-------+--------------------+-----------+------+-------------------+---------------+----+
only showing top 20 rows

In [0]:
%python
data_merged_sg_1=df_filter_1.join(df_user, df_filter_1.customer_id ==df_user.user_id).drop(df_user.user_id)

data_merged_sg_1.show()

+-----------+------------------+------------------+------------------+--------------------+-----------+------+-------------------+---------------+---+
customer_id| home_lat| home_lon|Zone_Neighbourhood| email|nationality|gender| registration_date|user_ent_status|cor|
+-----------+------------------+------------------+------------------+--------------------+-----------+------+-------------------+---------------+---+
 9212923|25.077522384615385| 55.13476307692308| MARSA DUBAI|yasmin.magdy.awad...| EG| F|2019-11-21 22:03:16| 2| AE|
 7453109|25.080040911764712|55.136533735294115| MARSA DUBAI|touchnpt27@gmail.com| GR| F|2019-03-25 15:31:23| 2| AE|
 8847492|25.083667666666667| 55.146413| MARSA DUBAI|hollywatson19@out...| GB| F|2019-10-01 14:54:02| 2| GB|
 3683305|25.076121333333333|55.141101555555544| MARSA DUBAI|amrelemam@hotmail...| EG| M|2017-12-12 12:18:16| 2| AE|
 368402|25.082325705882354| 55.14778517647058| MARSA DUBAI|scottgrayston@hot...| GB| M|2015-02-25 02:51:30| 2| AE|
 3506688| 25.07835052352941| 55.13565458235294| MARSA DUBAI|jackjohnsim@gmail...| GB| M|2017-11-13 21:46:50| 2| AE|
 9877935|25.083549799999997| 55.146479| MARSA DUBAI|germain.buendia@h...| MX| M|2020-01-27 21:20:54| 2| AE|
 9249261|25.076340849999998| 55.13910655000001| MARSA DUBAI|salimpierrot@hotm...| FR| M|2019-11-26 07:51:41| 2| AE|
 1925409| 25.0756445| 55.1310672| MARSA DUBAI|ahmed.mohamed.abd...| AE| M|2016-10-07 03:15:15| 2| AE|
 244755|25.086900321839074| 55.14788005747126| MARSA DUBAI|staceyscanlon23@g...| IE| F|2014-09-08 21:16:17| 2| AE|
 1295492| 25.0744810375| 55.13060182499999| MARSA DUBAI|brijeshmaurya2@gm...| IN| M|2016-04-09 13:37:32| 2| AE|
 3073099| 25.07795475| 55.13333741666667| MARSA DUBAI| chj8755@naver.com| KR| M|2017-07-23 11:13:26| 2| AE|
 525833|25.085988714285715| 55.14091785714286| MARSA DUBAI|claire.fletcher03...| ZA| F|2015-07-13 15:38:04| 2| AE|
 113426|25.076480475409834| 55.13233804918032| MARSA DUBAI|geoshieldjizan@gm...| SA| M|2013-12-09 13:43:08| 2| SA|
 6827665| 25.07741482222222| 55.14025193333333| MARSA DUBAI| nabile@dbc-g.com| FR| M|2019-01-25 19:35:37| 2| AE|
 2902383|25.076454688888887| 55.13953206666668| MARSA DUBAI|k_shafiq67@hotmai...| PK| M|2017-05-28 00:48:06| 2| AE|
 288690|25.083995061403506|55.144507315789475| MARSA DUBAI|firas.awad84@gmai...| SY| F|2014-11-29 15:41:31| 2| AE|
 330486| 25.0788934| 55.1353478| MARSA DUBAI| raynbrown@sky.com| GB| M|2015-01-14 11:44:39| 2| GB|
 3592800| 25.0663094| 55.13120310000001| MARSA DUBAI|ahmed.allam@dewa....| EG| M|2017-11-28 19:06:01| 2| AE|
 1488998|25.086781066666664| 55.15207426666667| MARSA DUBAI|almithal54@gmail.com| IN| M|2016-05-22 07:18:36| 2| AE|
+-----------+------------------+------------------+------------------+--------------------+-----------+------+-------------------+---------------+---+
only showing top 20 rows

In [0]:
%python
data_merged_sg_2=df_filter_2.join(df_user, df_filter_2.customer_id ==df_user.user_id).drop(df_user.user_id)

data_merged_sg_2.show()

+-----------+------------------+------------------+--------------------+--------------------+-----------+------+-------------------+---------------+---+
customer_id| home_lat| home_lon| Zone_Neighbourhood| email|nationality|gender| registration_date|user_ent_status|cor|
+-----------+------------------+------------------+--------------------+--------------------+-----------+------+-------------------+---------------+---+
 407764| 25.04407483333333| 55.25062633333334|AL HEBIAH SECOND ...|kani.saritas@yaho...| DE| M|2015-04-24 09:29:27| 2| AE|
 5328189|25.027306863636365| 55.23775404545454|AL HEBIAH SECOND ...|jonnyfisher20@gma...| GB| M|2018-08-02 08:30:39| 2| AE|
 30361| 25.04419692307692| 55.25056384615384|AL HEBIAH SECOND ...|michelle.mochan@g...| GB| F|2011-03-01 04:00:00| 2| AE|
 3284581|25.044269647058822| 55.25072417647059|AL HEBIAH SECOND ...|rachaelhelenaknow...| GB| F|2017-09-30 19:43:31| 2| AE|
 264326| 25.04411790909091|55.250742090909085|AL HEBIAH SECOND ...|folaaiyedun@yahoo...| NG| F|2014-10-14 18:56:44| 2| AE|
 8635090|25.042185664285718| 55.24707325357142|AL HEBIAH SECOND ...|paulrichardson82@...| GB| M|2019-08-26 23:55:42| 2| AE|
 159131| 25.0441884375|55.250535000000006|AL HEBIAH SECOND ...| heatherdrake@me.com| GB| F|2014-02-27 13:01:18| 2| AE|
 5422576| 25.04423188888889| 55.25050194444445|AL HEBIAH SECOND ...| katiemaher@live.com| GB| F|2018-08-22 17:05:25| 2| AE|
 9780316|25.042784937500002| 55.24765845|AL HEBIAH SECOND ...|dalia.ab@samsung.com| EG| F|2020-01-16 12:43:07| 2| AE|
 630943| 25.0432418125| 55.2489413125|AL HEBIAH SECOND ...| katesmith3@sky.com| GB| F|2015-09-26 15:03:50| 2| AE|
 10127517|25.044174879999996| 55.25068957999999|AL HEBIAH SECOND ...|rebecca@pineapple...| GB| F|2020-02-27 17:58:31| 2| AE|
 3461249|25.044034266666664| 55.25039386666666|AL HEBIAH SECOND ...|filippo.perta@gma...| IT| M|2017-11-05 20:18:05| 2| AE|
 9276166|25.044361045454547| 55.25079764545455|AL HEBIAH SECOND ...|sanjiv4380@gmail.com| IN| M|2019-11-29 12:24:49| 2| AE|
 728185| 25.04397613157895| 55.25056884210526|AL HEBIAH SECOND ...|neil.parkins@alfu...| GB| M|2015-11-29 08:54:29| 2| AE|
 5440444| 25.04403845| 55.25068115|AL HEBIAH SECOND ...| emboland@tcd.ie| IE| F|2018-08-26 12:36:17| 2| AE|
 22988| 25.0442561| 55.25057059999999|AL HEBIAH SECOND ...|hughesjo@hotmail....| GB| F|2011-03-01 04:00:00| 2| AE|
 9912632|25.044160333333334| 55.25068491666665|AL HEBIAH SECOND ...|bull.jordan22@gma...| GB| M|2020-02-01 13:41:18| 2| AE|
 6391590|25.044339833333336| 55.25077616666667|AL HEBIAH SECOND ...| DenK751@yandex.ru| RU| M|2019-01-05 09:03:02| 2| AE|
 317745| 25.04407157692308| 55.25065692307691|AL HEBIAH SECOND ...|amin.mostafa@gmai...| EG| M|2015-01-03 16:37:23| 2| AE|
 5863| 25.04420641666667| 55.25070958333333|AL HEBIAH SECOND ...| aamynah@gmail.com| IN| F|2012-12-18 03:04:03| 2| AE|
+-----------+------------------+------------------+--------------------+--------------------+-----------+------+-------------------+---------------+---+
only showing top 20 rows

In [0]:
data_merged_sg_2=data_merged_sg_2.drop('home_lat','home_lon','nationality','gender','registration_date','user_ent_status','cor')
data_merged_sg_2.show()

+-----------+--------------------+--------------------+
customer_id| Zone_Neighbourhood| email|
+-----------+--------------------+--------------------+
 407764|AL HEBIAH SECOND ...|kani.saritas@yaho...|
 5328189|AL HEBIAH SECOND ...|jonnyfisher20@gma...|
 30361|AL HEBIAH SECOND ...|michelle.mochan@g...|
 3284581|AL HEBIAH SECOND ...|rachaelhelenaknow...|
 264326|AL HEBIAH SECOND ...|folaaiyedun@yahoo...|
 8635090|AL HEBIAH SECOND ...|paulrichardson82@...|
 159131|AL HEBIAH SECOND ...| heatherdrake@me.com|
 5422576|AL HEBIAH SECOND ...| katiemaher@live.com|
 9780316|AL HEBIAH SECOND ...|dalia.ab@samsung.com|
 630943|AL HEBIAH SECOND ...| katesmith3@sky.com|
 10127517|AL HEBIAH SECOND ...|rebecca@pineapple...|
 3461249|AL HEBIAH SECOND ...|filippo.perta@gma...|
 9276166|AL HEBIAH SECOND ...|sanjiv4380@gmail.com|
 728185|AL HEBIAH SECOND ...|neil.parkins@alfu...|
 5440444|AL HEBIAH SECOND ...| emboland@tcd.ie|
 22988|AL HEBIAH SECOND ...|hughesjo@hotmail....|
 9912632|AL HEBIAH SECOND ...|bull.jordan22@gma...|
 6391590|AL HEBIAH SECOND ...| DenK751@yandex.ru|
 317745|AL HEBIAH SECOND ...|amin.mostafa@gmai...|
 5863|AL HEBIAH SECOND ...| aamynah@gmail.com|
+-----------+--------------------+--------------------+
only showing top 20 rows

In [0]:
data_merged_sg_1=data_merged_sg_1.drop('home_lat','home_lon','nationality','gender','registration_date','user_ent_status','cor')
data_merged_sg_1.show()

+-----------+------------------+--------------------+
customer_id|Zone_Neighbourhood| email|
+-----------+------------------+--------------------+
 9212923| MARSA DUBAI|yasmin.magdy.awad...|
 7453109| MARSA DUBAI|touchnpt27@gmail.com|
 8847492| MARSA DUBAI|hollywatson19@out...|
 3683305| MARSA DUBAI|amrelemam@hotmail...|
 368402| MARSA DUBAI|scottgrayston@hot...|
 3506688| MARSA DUBAI|jackjohnsim@gmail...|
 9877935| MARSA DUBAI|germain.buendia@h...|
 9249261| MARSA DUBAI|salimpierrot@hotm...|
 1925409| MARSA DUBAI|ahmed.mohamed.abd...|
 244755| MARSA DUBAI|staceyscanlon23@g...|
 1295492| MARSA DUBAI|brijeshmaurya2@gm...|
 3073099| MARSA DUBAI| chj8755@naver.com|
 525833| MARSA DUBAI|claire.fletcher03...|
 113426| MARSA DUBAI|geoshieldjizan@gm...|
 6827665| MARSA DUBAI| nabile@dbc-g.com|
 2902383| MARSA DUBAI|k_shafiq67@hotmai...|
 288690| MARSA DUBAI|firas.awad84@gmai...|
 330486| MARSA DUBAI| raynbrown@sky.com|
 3592800| MARSA DUBAI|ahmed.allam@dewa....|
 1488998| MARSA DUBAI|almithal54@gmail.com|
+-----------+------------------+--------------------+
only showing top 20 rows

In [0]:
data_merged_sg_1.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/marsa_dubai_list.csv")
data_merged_sg_2.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/studio_city_list.csv")


